<a href="https://colab.research.google.com/github/NoahOshana17/Fruit-Freshness-Classification/blob/main/fruit_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Dependencies**


In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt 
from PIL import Image
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
import numpy as np
import os

###**Checking Information**

In [ ]:
tf.test.gpu_device_name()

###**Setting Up Paths**

In [ ]:
data_dir_path = Path('/content/drive/MyDrive/Fruits2/test/freshapples')
train_path = Path('/content/drive/MyDrive/Fruits2/train/')
test_path = Path('/content/drive/MyDrive/Fruits2/test/')

###**Rename Images**

In [ ]:
for count, filename in enumerate(os.listdir(data_dir_path)):
  src = os.path.join(data_dir_path, filename)
  dst = os.path.join(data_dir_path, f'{str(count)}.png')
  os.rename(src, dst)


###**Display an Image for Testing Purposes**

In [ ]:
test_image = Path("/content/drive/MyDrive/Fruits2/train/freshoranges/692.png")

Image.open(test_image)

###**Basic Training Settings**

In [ ]:
im_height = 224
im_width = 224
batch_size = 32

###**Data Preprocessing**

In [ ]:
generator = ImageDataGenerator(
    # rotation_range=45,
    rescale=1./255,
    # horizontal_flip=True,
    # vertical_flip=True,
    validation_split=.2
)

train_generator = generator.flow_from_directory(
    train_path,
    target_size=(im_height, im_width),
    batch_size = batch_size,
    subset='training'
)

validation_generator = generator.flow_from_directory(
    train_path,
    target_size=(im_height, im_width),
    batch_size=batch_size,
    subset='validation'
)

test_generator = generator.flow_from_directory(
    test_path,
    target_size= (im_height, im_width),
    batch_size= batch_size,
    shuffle=False
)

###**Original MobileNetV2**

In [ ]:
model = MobileNetV2()
model.summary()
print(len(model.layers))


###**Fine-Tune MobileNetV2**

In [ ]:
x = model.layers[-2].output
output = Dense(units=6, activation='softmax')(x)

tuned_model = Model(inputs=model.input, outputs=output)
tuned_model.summary()
print(len(tuned_model.layers))

###**Visualize Model Architecture**

In [ ]:
plot_model(tuned_model, to_file='/content/drive/MyDrive/Model Architectures/tuned_model.png')

###**Train Model**

In [ ]:
tuned_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy',
                    metrics=['accuracy'])

history = tuned_model.fit(train_generator, validation_data=validation_generator,
                epochs=10, verbose=2)

### **Visualize Training History**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = 10

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


###**Save Model**

In [ ]:
tuned_model.save('/content/drive/MyDrive/SavedModel/tuned_mobilenetv2_tf', save_format='tf')

###**Load Model**

In [ ]:
tuned_model = tf.keras.models.load_model('/content/drive/MyDrive/SavedModel/tuned_mobilenetv2_tf')

###**Continue Training**

In [ ]:
tuned_model.fit(train_generator, validation_data=validation_generator, epochs=2, verbose=2)

###**Test Model**

In [ ]:
class_names = ['freshapples', 'freshbananas', 'freshoranges', 'rottenapples', 'rottenbananas', 'rottenoranges']

In [ ]:
predictions = tuned_model.predict(test_generator)
score = tf.nn.softmax(predictions[0])

print(
    'This image most likely belongs to {} with a {:.2f} percent confidence.'.format(
        class_names[np.argmax(score)], 100 * np.max(score)
    )
)

### **Test Model: Method 2**

>This needs the test generator to specify shuffle=False

---



In [ ]:
files=test_generator.filenames
class_dict=test_generator.class_indices # a dictionary of the form class name: class index
rev_dict={}
for key, value in class_dict.items():
    rev_dict[value]=key   # dictionary of the form class index: class name

In [ ]:
predictions = tuned_model.predict(test_generator)

In [ ]:
for i, p in enumerate(predictions):
    index=np.argmax(p)
    klass=rev_dict[index]    
    prob=p[index]
    print('for file ', files[i], ' predicted class is ', klass,' with probability ',prob)